In [1]:
import os
import cv2
import numpy as np
import shutil
import random
from pathlib import Path

# --- (No changes in these functions) ---

def rotate_image(image, angle):
    """Rotates the image by the given angle."""
    height, width = image.shape[:2]
    center = (width // 2, height // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))
    return rotated_image

def rotate_bbox(bbox, angle):
    """Rotates bounding box coordinates."""
    class_id, x_center, y_center, width, height = bbox
    if angle == 90:
        new_x, new_y = y_center, 1 - x_center
    elif angle == 180:
        new_x, new_y = 1 - x_center, 1 - y_center
    elif angle == 270:
        new_x, new_y = 1 - y_center, x_center
    else:
        return bbox
    return [class_id, new_x, new_y, width, height]

def flip_image(image):
    """Flips the image horizontally."""
    return cv2.flip(image, 1)

def flip_bbox(bbox):
    """Flips bounding box coordinates horizontally."""
    class_id, x_center, y_center, width, height = bbox
    new_x_center = 1 - x_center
    return [class_id, new_x_center, y_center, width, height]

# --- (Changes in this function) ---

def create_augmented_dataset():
    """Creates an augmented dataset (rotated, flipped images)."""
    base_path = Path("/kaggle/input/weed-detection-dataset/labeled")
    output_path = Path("/kaggle/working/augmented_dataset")

    # Create subdirectories for images and annotations
    (output_path / "images").mkdir(parents=True, exist_ok=True)
    (output_path / "annotations").mkdir(parents=True, exist_ok=True)

    for img_path in (base_path / "images").glob("*.jpg"):
        ann_path = base_path / "annotations" / f"{img_path.stem}.txt"
        if not ann_path.exists():
            continue

        # Copy original image and annotation
        shutil.copy2(img_path, output_path / "images" / img_path.name)
        shutil.copy2(ann_path, output_path / "annotations" / ann_path.name)

        # Read image and annotations
        image = cv2.imread(str(img_path))
        with open(ann_path, 'r') as f:
            bboxes = [list(map(float, line.strip().split())) for line in f]

        # --- Rotations ---
        for angle in [90, 180, 270]:
            new_img_name = f"{img_path.stem}_rot{angle}.jpg"
            new_ann_name = f"{img_path.stem}_rot{angle}.txt"
            
            rotated_image = rotate_image(image, angle)
            rotated_bboxes = [rotate_bbox(bbox, angle) for bbox in bboxes]
            
            cv2.imwrite(str(output_path / "images" / new_img_name), rotated_image)
            with open(output_path / "annotations" / new_ann_name, 'w') as f:
                for bbox in rotated_bboxes:
                    f.write(' '.join(map(str, bbox)) + '\n')

        # --- Flipping ---
        new_img_name = f"{img_path.stem}_flip.jpg"
        new_ann_name = f"{img_path.stem}_flip.txt"

        flipped_image = flip_image(image)
        flipped_bboxes = [flip_bbox(bbox) for bbox in bboxes]
        
        cv2.imwrite(str(output_path / "images" / new_img_name), flipped_image)
        with open(output_path / "annotations" / new_ann_name, 'w') as f:
            for bbox in flipped_bboxes:
                f.write(' '.join(map(str, bbox)) + '\n')

    print(f"✅ Augmented dataset saved in: {output_path}")

# --- (No changes in this function) ---

def restructure_dataset():
    """Moves the augmented dataset into Train, Dev, and Unlabeled folders."""
    ORIGINAL_DATASET_PATH = Path("/kaggle/input/weed-detection-dataset")
    AUGMENTED_DATASET_PATH = Path("/kaggle/working/augmented_dataset")
    OUTPUT_DATASET_PATH = Path("/kaggle/working/final")
    
    TRAIN_PATH = OUTPUT_DATASET_PATH / "Dataset" / "Train"
    DEV_PATH = OUTPUT_DATASET_PATH / "Dataset" / "Dev"
    UNLABELED_PATH = OUTPUT_DATASET_PATH / "Dataset" / "Unlabelled"
    
    # Create directories
    (TRAIN_PATH / "images").mkdir(parents=True, exist_ok=True)
    (TRAIN_PATH / "annotations").mkdir(parents=True, exist_ok=True)
    (DEV_PATH / "images").mkdir(parents=True, exist_ok=True)
    (DEV_PATH / "annotations").mkdir(parents=True, exist_ok=True)
    UNLABELED_PATH.mkdir(parents=True, exist_ok=True)
    
    # Move augmented data to Train
    for subdir in ["images", "annotations"]:
        src_path = AUGMENTED_DATASET_PATH / subdir
        dest_path = TRAIN_PATH / subdir
        for file in src_path.glob("*"):
            shutil.move(str(file), str(dest_path / file.name))
            
    # Copy test and unlabeled data
    shutil.copytree(ORIGINAL_DATASET_PATH / "test", DEV_PATH, dirs_exist_ok=True)
    shutil.copytree(ORIGINAL_DATASET_PATH / "unlabeled", UNLABELED_PATH, dirs_exist_ok=True)
    
    print("Dataset restructuring complete!")

# --- (Changes in this function) ---

def prepare_yolo_dataset():
    """Prepares YOLO dataset structure for training."""
    # Corrected the base directory path
    BASE_DIR = Path("/kaggle/working/final/Dataset")
    TRAIN_IMG_DIR = BASE_DIR / "Train" / "images"
    TRAIN_ANN_DIR = BASE_DIR / "Train" / "annotations"
    DEV_IMG_DIR = BASE_DIR / "Dev" / "images"
    DEV_ANN_DIR = BASE_DIR / "Dev" / "annotations"
    
    NEW_DATASET_DIR = Path("/kaggle/working/yolo_dataset")
    IMG_TRAIN_DIR = NEW_DATASET_DIR / "images" / "train"
    IMG_VAL_DIR = NEW_DATASET_DIR / "images" / "val"
    LBL_TRAIN_DIR = NEW_DATASET_DIR / "labels" / "train"
    LBL_VAL_DIR = NEW_DATASET_DIR / "labels" / "val"
    
    for folder in [IMG_TRAIN_DIR, IMG_VAL_DIR, LBL_TRAIN_DIR, LBL_VAL_DIR]:
        folder.mkdir(parents=True, exist_ok=True)
        
    image_files = sorted(list(TRAIN_IMG_DIR.glob("*.jpg"))) + sorted(list(DEV_IMG_DIR.glob("*.jpg")))
    random.seed(42)
    random.shuffle(image_files)
    
    split_idx = int(0.8 * len(image_files))
    train_files = image_files[:split_idx]
    val_files = image_files[split_idx:]
    
    def move_files(image_list, img_dest, lbl_dest, src_img_dir, src_lbl_dir):
        for img_path in image_list:
            img_name = img_path.name
            lbl_name = img_name.replace(".jpg", ".txt")
            lbl_path = Path(src_lbl_dir) / lbl_name
            if lbl_path.exists():
                shutil.copy(img_path, img_dest)
                shutil.copy(lbl_path, lbl_dest)

    # Corrected the source directories for the val_files
    move_files(train_files, IMG_TRAIN_DIR, LBL_TRAIN_DIR, TRAIN_IMG_DIR, TRAIN_ANN_DIR)
    move_files(val_files, IMG_VAL_DIR, LBL_VAL_DIR, TRAIN_IMG_DIR, TRAIN_ANN_DIR)
    move_files(train_files, IMG_TRAIN_DIR, LBL_TRAIN_DIR, DEV_IMG_DIR, DEV_ANN_DIR)
    move_files(val_files, IMG_VAL_DIR, LBL_VAL_DIR, DEV_IMG_DIR, DEV_ANN_DIR)
    
    # Corrected the paths in the YAML file
    yaml_content = f"""
    train: {IMG_TRAIN_DIR.resolve()}
    val: {IMG_VAL_DIR.resolve()}
    nc: 2
    names: ['Crop', 'Weed']
    """
    
    with open(NEW_DATASET_DIR / "data.yaml", "w") as f:
        f.write(yaml_content)
        
    print("YOLO dataset ready for training.")

# --- (No changes in the main block) ---

if __name__ == "__main__":
    create_augmented_dataset()
    restructure_dataset()
    prepare_yolo_dataset()
    print("All dataset preparation steps completed successfully!")

✅ Augmented dataset saved in: /kaggle/working/augmented_dataset
Dataset restructuring complete!
YOLO dataset ready for training.
All dataset preparation steps completed successfully!


In [2]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.1 MB/s eta 0:00:00:00:0100:01
  Attempting unins

In [3]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

True
Tesla T4


In [5]:
from ultralytics import YOLO

model = YOLO("yolo11m.pt")
model.train(data="/kaggle/working/yolo_dataset/data.yaml", epochs=50, imgsz=640, batch=16, device="cuda")
model.save("v11m_50_epochs.pt")

Ultralytics 8.3.203 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        210        339      0.835      0.813      0.879      0.538
                  Crop         92        174      0.856      0.874       0.91      0.558
                  Weed        118        165      0.813      0.752      0.847      0.518
Speed: 0.2ms preprocess, 12.2ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to /kaggle/working/runs/detect/train2


In [6]:
import torchvision.transforms as T
import torch
import os
import cv2
from ultralytics import YOLO

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = YOLO("v11m_50_epochs.pt")
model.model.to(device)

weak_transform = T.Compose([
    T.RandomHorizontalFlip(p=0.5),
    T.RandomResizedCrop(size=640, scale=(0.8, 1.0))
])

strong_transform = T.Compose([
    T.RandomHorizontalFlip(p=0.5),
    T.RandomResizedCrop(size=640, scale=(0.5, 1.0)),
    T.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    T.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 2.0))
])

UNLABELED_DIR = "/kaggle/working/final/Dataset/Unlabelled"
image_files = [f for f in os.listdir(UNLABELED_DIR) if f.endswith(".jpg")]

optimizer = torch.optim.Adam(model.model.parameters(), lr=5e-5)
for img_file in image_files:
    img_path = os.path.join(UNLABELED_DIR, img_file)
    img = cv2.imread(img_path)
    img_pil = T.ToPILImage()(img)

    weak_img = weak_transform(img_pil)
    strong_img = strong_transform(img_pil)
    weak_tensor = T.ToTensor()(weak_img).unsqueeze(0).to(device).requires_grad_(True)
    strong_tensor = T.ToTensor()(strong_img).unsqueeze(0).to(device).requires_grad_(True)

    model.model.to(device)
    weak_preds = model.model(weak_tensor)  # Get raw output logits
    strong_preds = model.model(strong_tensor)

    if isinstance(weak_preds, tuple):
        weak_preds = weak_preds[0]
        strong_preds = strong_preds[0]

    #Consistency loss
    loss = torch.nn.functional.mse_loss(weak_preds, strong_preds)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

model.save("model_consistency.pt")

In [8]:
import os
import torch
import cv2
import torchvision.transforms as T
from ultralytics import YOLO

model = YOLO("model_consistency.pt")
UNLABELED_DIR = "/kaggle/working/final/Dataset/Unlabelled"
PSEUDO_LABELS_DIR = "/kaggle/working/yolo_dataset/labels/pseudo"
os.makedirs(PSEUDO_LABELS_DIR, exist_ok=True)

results = model.predict(source=UNLABELED_DIR, save=False)

for i, result in enumerate(results):
    img_name = os.path.basename(result.path).replace(".jpg", ".txt")
    label_path = os.path.join(PSEUDO_LABELS_DIR, img_name)
    with open(label_path, "w") as f:
        for box in result.boxes.data:
            cls, x_center, y_center, width, height, conf = box.tolist()
            if conf > 0.85:
                f.write(f"{int(cls)} {x_center} {y_center} {width} {height}\n")

TRAIN_IMAGES_DIR = "/kaggle/working/yolo_dataset/images/train"
TRAIN_LABELS_DIR = "/kaggle/working/yolo_dataset/labels/train"

for file in os.listdir(PSEUDO_LABELS_DIR):
    if file.endswith(".txt"):
        shutil.copy(os.path.join(PSEUDO_LABELS_DIR, file), TRAIN_LABELS_DIR)
model.train(data="/kaggle/working/yolo_dataset/data.yaml", epochs=50, imgsz=640, batch=16, device="cuda")

model.save("model_pseudo.pt")


image 1/1000 /kaggle/working/final/Dataset/Unlabelled/agri_0_1009_jpeg.rf.bd6418200779c7138320a9dd87b37a01.jpg: 640x640 1 Weed, 36.9ms
image 2/1000 /kaggle/working/final/Dataset/Unlabelled/agri_0_1017_jpeg.rf.10421009c95805e81d82a2b6a8c3b116.jpg: 640x640 1 Weed, 36.9ms
image 3/1000 /kaggle/working/final/Dataset/Unlabelled/agri_0_1018_jpeg.rf.30238c6d22d384618e1ef4297dea6e34.jpg: 640x640 1 Weed, 36.9ms
image 4/1000 /kaggle/working/final/Dataset/Unlabelled/agri_0_1020_jpeg.rf.26add55029735d0e6ae4ec0a57bb210a.jpg: 640x640 1 Weed, 36.9ms
image 5/1000 /kaggle/working/final/Dataset/Unlabelled/agri_0_1024_jpeg.rf.f163741f6c70d41a5a88d6f1297d19a8.jpg: 640x640 1 Weed, 26.4ms
image 6/1000 /kaggle/working/final/Dataset/Unlabelled/agri_0_1026_jpeg.rf.e236d59f4d71d7c774ec9d53285b56f7.jpg: 640x640 1 Crop, 23.6ms
image 7/1000 /kaggle/working/final/Dataset/Unlabelled/agri_0_1028_jpeg.rf.32fc950fe95beacd547893f9915ab5a9.jpg: 640x640 14 Crops, 23.8ms
image 8/1000 /kaggle/working/final/Dataset/Unlabelle

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        210        339      0.855      0.833      0.894      0.575
                  Crop         92        174      0.893      0.902      0.925      0.607
                  Weed        118        165      0.818      0.764      0.863      0.543
Speed: 0.2ms preprocess, 12.2ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to /kaggle/working/runs/detect/train4


In [11]:
import os
import shutil
from ultralytics import YOLO

# --- Part 1: Generate High-Confidence Pseudo-Labels ---

# Load the model trained on the initial pseudo-labels
model = YOLO("model_pseudo.pt") 

# Define paths
UNLABELED_DIR = "/kaggle/working/final/Dataset/Unlabelled"
PSEUDO_LABELS_DIR = "/kaggle/working/yolo_dataset/labels/pseudo_fixmatch" # Use a new directory
os.makedirs(PSEUDO_LABELS_DIR, exist_ok=True)

# Predict on unlabeled images (this is for INFERENCE to get labels, not for training)
results = model.predict(source=UNLABELED_DIR, save=False, verbose=False)

# Save high-confidence predictions as new pseudo-labels
# We can use a stricter confidence threshold here as the model has improved
for result in results:
    img_name = os.path.basename(result.path).replace(".jpg", ".txt")
    label_path = os.path.join(PSEUDO_LABELS_DIR, img_name)
    with open(label_path, "w") as f:
        for box in result.boxes.data:
            cls, x_center, y_center, width, height, conf = box.tolist()
            if conf > 0.90:  # Using a stricter confidence for the second round
                f.write(f"{int(cls)} {x_center} {y_center} {width} {height}\n")

print(f"✅ FixMatch pseudo-labeling complete. Labels saved in: {PSEUDO_LABELS_DIR}")

# --- Part 2: Retrain with the New, Higher-Quality Pseudo-Labels ---

# Add the new pseudo-labels to the main training labels directory
TRAIN_LABELS_DIR = "/kaggle/working/yolo_dataset/labels/train"
for file in os.listdir(PSEUDO_LABELS_DIR):
    if file.endswith(".txt"):
        shutil.copy(os.path.join(PSEUDO_LABELS_DIR, file), TRAIN_LABELS_DIR)

print("✅ New pseudo-labels have been added to the training set.")

# Retrain the model using the combined dataset
model.train(
    data="/kaggle/working/yolo_dataset/data.yaml", 
    epochs=50, 
    imgsz=640, 
    batch=16, 
    device="cuda"
)

# Save the newly trained FixMatch model
model.save("model_fixmatch.pt")

print("✅ FixMatch model training complete! Weights saved to model_fixmatch.pt")

✅ FixMatch pseudo-labeling complete. Labels saved in: /kaggle/working/yolo_dataset/labels/pseudo_fixmatch
✅ New pseudo-labels have been added to the training set.
Ultralytics 8.3.203 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=model_pseudo.pt, momentum=0.937,

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        210        339      0.872      0.851      0.917      0.603
                  Crop         92        174      0.905      0.908       0.95      0.647
                  Weed        118        165      0.838      0.794      0.884       0.56
Speed: 0.2ms preprocess, 12.2ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to /kaggle/working/runs/detect/train5
✅ FixMatch model training complete! Weights saved to model_fixmatch.pt


In [12]:
import torch
import os
import shutil
from copy import deepcopy
from ultralytics import YOLO

# Load Student Model
student_model = YOLO("model_fixmatch.pt").to("cuda")
student_model.train()

# Initialize Teacher Model as an EMA of Student
teacher_model = deepcopy(student_model)
teacher_model.eval()

# Ensure pseudo-label directory exists
PSEUDO_LABELS_DIR = "/kaggle/working/labels/pseudo/"
os.makedirs(PSEUDO_LABELS_DIR, exist_ok=True)

# Define optimizer for student model
optimizer = torch.optim.AdamW(student_model.model.parameters(), lr=5e-5)

# EMA Decay Factor
alpha = 0.999

# Load Unlabeled Data
UNLABELED_DIR = "/kaggle/input/weed-detection-dataset/unlabeled"
image_files = [f for f in os.listdir(UNLABELED_DIR) if f.endswith(".jpg")]

# Train Student Model with EMA Teacher
for img_file in image_files:
    img_path = os.path.join(UNLABELED_DIR, img_file)

    # Get pseudo-labels from Teacher Model
    teacher_preds = teacher_model.predict(img_path, conf=0.85)

    if len(teacher_preds) == 0:
        continue  # Skip if no detections

    teacher_preds = teacher_preds[0]  # Extract first result

    # Convert pseudo-labels to training format
    label_file = os.path.join(PSEUDO_LABELS_DIR, img_file.replace(".jpg", ".txt"))
    with open(label_file, "w") as f:
        for i in range(len(teacher_preds.boxes.xywhn)):
            x_center, y_center, width, height = teacher_preds.boxes.xywhn[i].tolist()
            cls = int(teacher_preds.boxes.cls[i].item())
            f.write(f"{cls} {x_center} {y_center} {width} {height}\n")

print("✅ Pseudo-label generation completed.")

# Retrain the student model using pseudo-labels
student_model.train(data="/kaggle/working/yolo_dataset/data.yaml", epochs=50, imgsz=640, batch=16, device="cuda")

# Update Teacher Model with EMA after training
for teacher_param, student_param in zip(teacher_model.model.parameters(), student_model.model.parameters()):
    teacher_param.data = alpha * teacher_param.data + (1 - alpha) * student_param.data

# Save the Mean Teacher Student Model
student_model.save("model_mean_teacher.pt")

print("✅ Mean Teacher Model training completed and saved as 'model_mean_teacher.pt'.")

engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=model_fixmatch.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train6, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=None, rect=False, resu

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        210        339      0.876      0.857      0.919      0.641
                  Crop         92        174      0.912      0.914      0.961      0.707
                  Weed        118        165      0.841        0.8      0.877      0.575
Speed: 0.2ms preprocess, 12.1ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to /kaggle/working/runs/detect/train6


NotImplementedError: ('{} cannot be pickled', '_MultiProcessingDataLoaderIter')

In [13]:
student_model.save("model_trained.pt") 

In [15]:
import torch
from ultralytics import YOLO
from tabulate import tabulate  # For nice tables

# List your trained models
models_info = [
    {"name": "model 1", "path": "/kaggle/working/model_consistency.pt"},
    {"name": "Model2", "path": "/kaggle/working/model_fixmatch.pt"},
    {"name": "Model3", "path": "/kaggle/working/model_pseudo.pt"},
    {"name": "Model4", "path": "/kaggle/working/model_trained.pt"},
    # Add more models here if needed
]

# Path to your dataset YAML
data_yaml = "/kaggle/working/yolo_dataset/data.yaml"

# List to store metrics for comparison
comparison_metrics = []

for info in models_info:
    print(f"\nEvaluating {info['name']}...")
    model = YOLO(info["path"]).to("cuda")
    model.eval()
    
    # Run validation
    results = model.val(data=data_yaml)
    res_dict = results.results_dict
    
    # Extract metrics safely
    precision = res_dict.get('metrics/precision(B)', 0)
    recall = res_dict.get('metrics/recall(B)', 0)
    mAP_50_95 = res_dict.get('metrics/mAP50-95(B)', 0)
    
    # Compute F1
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    combined_metric = 0.5 * f1_score + 0.5 * mAP_50_95
    
    comparison_metrics.append({
        "Model": info["name"],
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1_score,
        "mAP50-95": mAP_50_95,
        "Combined": combined_metric
    })

# Print comparative table
print("\n=== Comparative Analysis ===")
print(tabulate(comparison_metrics, headers="keys", tablefmt="fancy_grid", floatfmt=".4f"))



Evaluating model 1...
YOLO11m summary (fused): 125 layers, 20,031,574 parameters, 0 gradients, 67.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2429.0±807.2 MB/s, size: 153.4 KB)
val: Scanning /kaggle/working/yolo_dataset/labels/val.cache... 210 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 210/210 446.4Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 14/14 1.8it/s 7.7s0.5s


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        210        339      0.837      0.812      0.879       0.54
                  Crop         92        174      0.858      0.871      0.912      0.559
                  Weed        118        165      0.816      0.754      0.847       0.52
Speed: 1.0ms preprocess, 30.1ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /kaggle/working/runs/detect/val2

Evaluating Model2...
YOLO11m summary (fused): 125 layers, 20,031,574 parameters, 0 gradients, 67.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2592.0±588.9 MB/s, size: 128.5 KB)
val: Scanning /kaggle/working/yolo_dataset/labels/val.cache... 210 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 210/210 400.5Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 14/14 1.8it/s 7.9s0.5s


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        210        339      0.874      0.851      0.916        0.6
                  Crop         92        174      0.907      0.908       0.95      0.643
                  Weed        118        165      0.841      0.794      0.883      0.557
Speed: 1.6ms preprocess, 30.5ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to /kaggle/working/runs/detect/val3

Evaluating Model3...
YOLO11m summary (fused): 125 layers, 20,031,574 parameters, 0 gradients, 67.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2250.0±573.4 MB/s, size: 120.0 KB)
val: Scanning /kaggle/working/yolo_dataset/labels/val.cache... 210 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 210/210 536.1Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 14/14 1.8it/s 7.8s0.5s


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        210        339      0.855      0.833      0.892      0.574
                  Crop         92        174      0.893      0.902      0.925      0.607
                  Weed        118        165      0.818      0.764       0.86      0.541
Speed: 1.8ms preprocess, 29.2ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /kaggle/working/runs/detect/val4

Evaluating Model4...
YOLO11m summary (fused): 125 layers, 20,031,574 parameters, 0 gradients, 67.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2032.5±708.5 MB/s, size: 147.3 KB)
val: Scanning /kaggle/working/yolo_dataset/labels/val.cache... 210 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 210/210 432.6Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 14/14 1.9it/s 7.3s0.5s


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        210        339       0.89       0.86      0.921      0.642
                  Crop         92        174      0.923      0.914      0.961      0.707
                  Weed        118        165      0.856      0.806      0.882      0.576
Speed: 2.1ms preprocess, 27.9ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /kaggle/working/runs/detect/val5

=== Comparative Analysis ===
╒═════════╤═════════════╤══════════╤════════════╤════════════╤════════════╕
│ Model   │   Precision │   Recall │   F1-Score │   mAP50-95 │   Combined │
╞═════════╪═════════════╪══════════╪════════════╪════════════╪════════════╡
│ model 1 │      0.8373 │   0.8124 │     0.8247 │     0.5395 │     0.6821 │
├─────────┼─────────────┼──────────┼────────────┼────────────┼────────────┤
│ Model2  │      0.8738 │   0.8510 │     0.8622 │     0.5998 │     0.7310 │
├─────────┼─────────────┼──────────┼────────────┼────────────┼────────────┤
│ Model3  │      0.8553 │   0.8330 │  